[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/03_Audit_Trail_and_Report.ipynb)

# Audit Trail and Clinical Report

**Goal:** export audit logs + PDF report from a run.


In [ ]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


In [ ]:
import json
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


In [ ]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


In [ ]:
outputs = iints.run_full(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=21,
    output_dir="results/audit_report",
)

outputs["audit"]


## Read the audit summary


In [ ]:
with open(outputs["audit"]["summary"], "r") as handle:
    summary = json.load(handle)
summary


## Report location


In [ ]:
outputs.get("report_pdf")


### Recap
Audit logs are in JSONL/CSV, and the PDF sits beside them.
